In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from pycbc import frame
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.waveform import get_fd_waveform, get_td_waveform
from pycbc.filter import highpass_fir, matched_filter, match
from pycbc.vetoes import power_chisq_bins, power_chisq
from pycbc.events import newsnr
%matplotlib inline

In [ ]:
hstrain = frame.read_frame('../Data/H-H1_LOSC_4_V1-1126076416-4096.gwf', 'H1:LOSC-STRAIN')
lstrain = frame.read_frame('../Data/L-L1_LOSC_4_V1-1126076416-4096.gwf', 'L1:LOSC-STRAIN')

In [ ]:
hp, hc = get_td_waveform(approximant="IMRPhenomD", mass1=20, mass2=20, delta_t=hstrain.delta_t, f_lower=30, distance = 1500)
fp, fc = get_fd_waveform(approximant="IMRPhenomD", mass1=20, mass2=20, delta_f=hstrain.delta_f, f_lower=30, distance = 1500)
pp = np.abs(fp*np.conj(fp))

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(hp.sample_times, hp)
plt.xlim([-0.5, 0.1])
plt.xlabel('Time - seconds')
plt.ylabel('Strain Amplitude')
plt.grid()
plt.savefig('wave.png')
plt.figure(figsize=(16, 8))
plt.plot(pp.sample_frequencies, pp)
plt.xlim([fp.sample_frequencies[0], 500])
plt.xlabel('Frequency - hertz')
plt.ylabel('Power')
plt.grid()
plt.savefig('freq.png')

In [ ]:
hpsd = hstrain.psd(4)
lpsd = lstrain.psd(4)

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('PSD')
plt.plot(hpsd.sample_frequencies, hpsd, label='Hanford', c='cornflowerblue')
plt.plot(lpsd.sample_frequencies, lpsd, label='Livingston', c='crimson')
plt.xscale('log')
plt.yscale('log')
plt.xlim([10, 1500])
plt.ylim([1e-47, 1e-38])
plt.xlabel('Frequency - hertz')
plt.ylabel('Power')
plt.legend()
plt.grid()
plt.savefig('psd.png')

In [ ]:
hwhite = hstrain.whiten(4, 4)

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Strain over 1 second')
plt.plot(hstrain.sample_times, hstrain)
plt.xlim([hstrain.sample_times[0] + 100, hstrain.sample_times[0] + 101])
plt.ylim([-0.5*10**-18, 0.5*10**-18])
plt.xlabel('GPS Time')
plt.ylabel('Strain')
plt.grid()
plt.savefig('strain.png')
plt.figure(figsize=(16, 8))
plt.title('Whitened strain over 1 second')
plt.plot(hwhite.sample_times, hwhite)
plt.xlim([hstrain.sample_times[0] + 100, hstrain.sample_times[0] + 101])
plt.ylim([-200, 200])
plt.xlabel('GPS Time')
plt.ylabel('Strain')
plt.grid()
plt.savefig('white.png')

In [ ]:
hpsd = interpolate(hpsd, hstrain.delta_f)
hpsd = inverse_spectrum_truncation(hpsd, hstrain.sample_rate * 4, low_frequency_cutoff=15)

temp = hp.copy()
temp.resize(len(hstrain))
temp = temp.cyclic_time_shift(temp.start_time)

snr = np.abs(matched_filter(temp, hstrain, psd=hpsd, low_frequency_cutoff=20))
gt = snr.sample_times[np.argmax(snr[16*snr.sample_rate:len(snr)-16*snr.sample_rate])]
gi = np.argmax(hwhite.sample_times > gt)

t, f, q = hwhite[gi-int(hwhite.sample_rate*1):gi+int(hwhite.sample_rate*1)].qtransform(hp.delta_t, logfsteps=500, frange=(30, 1000))

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Strain time-series')
plt.plot(hwhite.sample_times, hwhite)
plt.xlim([gt - 0.5, gt + 0.5])
plt.xlabel('GPS Time')
plt.ylabel('Strain')
plt.grid()
plt.savefig('glitch.png')
plt.figure(figsize=(16, 8))
plt.title('SNR time-series')
plt.plot(snr.sample_times, snr)
plt.xlim([gt - 0.5, gt + 0.5])
plt.ylim([0, 10])
plt.xlabel('GPS Time')
plt.ylabel('SNR')
plt.grid()
plt.savefig('glitch_snr.png')
plt.figure(figsize=(16, 8))
plt.title('Q-Scan')
plt.xlabel('GPS Time')
plt.ylabel('Frequency - hertz')
pylab.pcolormesh(t, f, q**0.5)
pylab.yscale('log')
plt.savefig('glitch_q.png')

In [ ]:
hhstrain = hstrain.copy()
hhstrain += temp.cyclic_time_shift(32).numpy()
hhwhite = hhstrain.whiten(4, 4)

hsnr = np.abs(matched_filter(temp, hhstrain, psd=hpsd, low_frequency_cutoff=20))
st = hhstrain.sample_times[32*hhstrain.sample_rate]
si = np.argmax(hhwhite.sample_times > st)

ht, hf, hq = hhwhite[si-int(hhwhite.sample_rate*1):si+int(hhwhite.sample_rate*1)].qtransform(hp.delta_t, logfsteps=500, frange=(30, 1000))

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Strain time-series')
plt.plot(hhwhite.sample_times, hhwhite)
plt.xlim([st - 0.5, st + 0.5])
plt.ylim([-200, 200])
plt.xlabel('GPS Time')
plt.ylabel('Strain')
plt.grid()
plt.savefig('cbc.png')
plt.figure(figsize=(16, 8))
plt.title('SNR time-series')
plt.plot(hsnr.sample_times, hsnr)
plt.xlim([st - 0.5, st + 0.5])
plt.ylim([0, 10])
plt.xlabel('GPS Time')
plt.ylabel('SNR')
plt.grid()
plt.savefig('cbc_snr.png')
plt.figure(figsize=(16, 8))
plt.title('Q-Scan')
plt.xlabel('GPS Time')
plt.ylabel('Frequency - hertz')
pylab.pcolormesh(ht, hf, hq**0.5)
pylab.yscale('log')
plt.savefig('cbc_q.png')

In [ ]:
nbins = 3
tf = temp.to_frequencyseries()
bins = power_chisq_bins(tf, nbins, hpsd, low_frequency_cutoff=20)
print(bins)

In [ ]:
c = ['cornflowerblue', 'crimson', 'darkgreen']
l = ['low freq bin', 'mid freq bin', 'high freq bin']
plt.figure(figsize=[16, 8])
for i in range(nbins):
    pf = tf[bins[i]:bins[i+1]]
    plt.plot(tf.sample_frequencies[bins[i]:bins[i+1]], pf*np.conj(pf), c=c[i])
    plt.xlim([0, 500])
    plt.xlabel('Frequency - hertz')
    plt.ylabel('Power')
    plt.grid()
plt.savefig('bins.png')

In [ ]:
chi, bsnr = power_chisq(temp, hhstrain, nbins, hpsd, low_frequency_cutoff=20, return_bins=True)

In [ ]:
plt.figure(figsize=[16, 8])
for i in range(nbins):
    bt = hhstrain.sample_times[32*hhstrain.sample_rate]
    bi = np.argmax(bsnr[i].sample_times > st)
    #plt.subplot(nbins, 1, i+1)
    plt.plot(bsnr[i].sample_times, np.abs(bsnr[i]), c=c[i], label=l[i])
    plt.xlim([bt - 0.1, bt + 0.1])
    plt.ylim(0, 7)
    plt.xlabel('GPS Time')
    plt.ylabel('SNR')
    plt.grid()
    plt.legend()
plt.savefig('cbc_bins.png')

In [ ]:
plt.figure(figsize=[16, 8])
for i in range(nbins):
    bt = hhstrain.sample_times[32*hhstrain.sample_rate]
    bi = np.argmax(bsnr[i].sample_times > st)
    #plt.subplot(nbins, 1, i+1)
    plt.plot(bsnr[i].sample_times, np.abs(bsnr[i]), c=c[i], label=l[i])
    plt.xlim([gt - 0.1, gt + 0.1])
    plt.ylim(0, 15)
    plt.xlabel('GPS Time')
    plt.ylabel('SNR')
    plt.grid()
    plt.legend()
plt.savefig('glitch_bins.png')

In [ ]:
newsnr = newsnr(hsnr, chi, q=nbins)

plt.figure(figsize=(16, 8))
plt.title('SNR time-series')
plt.plot(hsnr.sample_times, hsnr, label='SNR')
plt.plot(chi.sample_times, chi, label='chi squared')
plt.plot(hsnr.sample_times, newsnr, label='new SNR')
plt.xlim([st - 0.5, st + 0.5])
plt.xlabel('GPS Time')
plt.ylabel('SNR')
plt.grid()
plt.legend()
plt.savefig('cbc_new.png')
plt.figure(figsize=(16, 8))
plt.title('SNR time-series')
plt.plot(hsnr.sample_times, hsnr, label='SNR')
plt.plot(chi.sample_times, chi, label='chi squared')
plt.plot(newsnr.sample_times, newsnr, label='new SNR')
plt.xlim([gt - 0.5, gt + 0.5])
plt.xlabel('GPS Time')
plt.ylabel('SNR')
plt.grid()
plt.legend()
plt.savefig('glitch_new.png')